# Geocoding of schools dataset

In [58]:
import pandas as pd
import requests
import os

In [65]:
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

In [2]:
folder_path = '../data/raw'
primary_schools = 'mainstream-schools_dublin_2019-2020.xlsx'
secondary_schools = 'post-primary-schools-2019-2020-1.xlsx'
special_schools = 'special-schools_dublin_2019-2020.xlsx'

In [3]:
data = pd.ExcelFile(f"{folder_path}/{primary_schools}")

In [4]:
data.sheet_names

['Sheet1']

In [5]:
df = data.parse('Sheet1')

In [6]:
def extract_lat_long_via_address(address_or_zipcode):
    lat, lng = None, None
    api_key = GOOGLE_API_KEY
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    endpoint = f"{base_url}?address={address_or_zipcode}&key={api_key}"
    # see how our endpoint includes our API key? Yes this is yet another reason to restrict the key
    r = requests.get(endpoint)
    if r.status_code not in range(200, 299):
        return None, None
    try:
        '''
        This try block incase any of our inputs are invalid. This is done instead
        of actually writing out handlers for all kinds of responses.
        '''
        results = r.json()['results'][0]
        lat = results['geometry']['location']['lat']
        lng = results['geometry']['location']['lng']
    except:
        pass
    return (lat, lng)

In [51]:
df['appended_address'] = df.Eircode.fillna(df.iloc[:,1] + ' ' + df.iloc[:,2] + ', ' + df.iloc[:,3] + ', ' + df.iloc[:,4])

In [ ]:
df['lat_long'] = df.apply (lambda row: extract_lat_long_via_address(row.appended_address), axis=1)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))